In [79]:
# Perform initial import of the needed libraries

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import sklearn
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from IPython.display import display

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [43]:
# Read the CSV and perform basic data cleaning
file_path = Path('./Resources/fighters+fights_export.csv')
fnf_df = pd.read_csv(file_path)[:-2]

In [44]:
fnf_df.columns

Index(['fight_id', 'winner', 'title_bout', 'weight_class', 'no_of_rounds',
       'r_fighter', 'r_fighter_id', 'r_fighter_stance', 'b_fighter',
       'b_fighter_id', 'b_fighter_stance', 'date', 'referee', 'referee_id',
       'b_fighter_height', 'r_fighter_height', 'b_fighter_reach',
       'r_fighter_reach', 'b_fighter_weight', 'r_fighter_weight'],
      dtype='object')

In [65]:
columns = (['winner', 'title_bout', 'weight_class', 'no_of_rounds', 'r_fighter_stance', 'b_fighter_stance', 'b_fighter_height', 'r_fighter_height', 'b_fighter_reach',
       'r_fighter_reach', 'b_fighter_weight', 'r_fighter_weight'])

target = ["winner"]

In [66]:
fnf_df = fnf_df.loc[:, columns].copy()

In [67]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,183.289714,183.665789,216.0,175.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,183.665789,205.0,175.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,183.289714,183.665789,196.0,175.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,183.289714,183.665789,185.0,190.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,183.289714,183.665789,275.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5137,Blue,FALSE,Women's Strawweight,3,Orthodox,Orthodox,165.10,160.02,167.640000,162.560000,115.0,115.0
5138,Red,TRUE,Women's Flyweight,5,Southpaw,Orthodox,167.64,165.10,167.640000,167.640000,125.0,125.0
5139,Red,FALSE,Women's Strawweight,3,unlisted,Orthodox,165.10,165.10,162.560000,167.640000,115.0,115.0
5140,Red,FALSE,Women's Flyweight,3,Orthodox,Orthodox,167.64,175.26,165.100000,172.720000,125.0,125.0


In [68]:
fnf_df.isnull().sum()

winner              0
title_bout          0
weight_class        0
no_of_rounds        0
r_fighter_stance    0
b_fighter_stance    0
b_fighter_height    0
r_fighter_height    0
b_fighter_reach     0
r_fighter_reach     0
b_fighter_weight    0
r_fighter_weight    0
dtype: int64

In [69]:
fnf_df['r_fighter_stance'].fillna("unlisted", inplace = True)
fnf_df['b_fighter_stance'].fillna("unlisted", inplace = True)
fnf_df['r_fighter_height'].fillna(value=fnf_df['r_fighter_height'].mean(), inplace = True)
fnf_df['b_fighter_height'].fillna(value=fnf_df['b_fighter_height'].mean(), inplace = True)
fnf_df['b_fighter_reach'].fillna(value=fnf_df['b_fighter_reach'].mean(), inplace = True)
fnf_df['r_fighter_reach'].fillna(value=fnf_df['r_fighter_reach'].mean(), inplace = True)
fnf_df['b_fighter_weight'].fillna(value=fnf_df['b_fighter_weight'].mean(), inplace = True)
fnf_df['r_fighter_weight'].fillna(value=fnf_df['r_fighter_weight'].mean(), inplace = True)

In [70]:
fnf_df.isnull().sum()

winner              0
title_bout          0
weight_class        0
no_of_rounds        0
r_fighter_stance    0
b_fighter_stance    0
b_fighter_height    0
r_fighter_height    0
b_fighter_reach     0
r_fighter_reach     0
b_fighter_weight    0
r_fighter_weight    0
dtype: int64

In [71]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,183.289714,183.665789,216.0,175.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,183.665789,205.0,175.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,183.289714,183.665789,196.0,175.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,183.289714,183.665789,185.0,190.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,183.289714,183.665789,275.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5137,Blue,FALSE,Women's Strawweight,3,Orthodox,Orthodox,165.10,160.02,167.640000,162.560000,115.0,115.0
5138,Red,TRUE,Women's Flyweight,5,Southpaw,Orthodox,167.64,165.10,167.640000,167.640000,125.0,125.0
5139,Red,FALSE,Women's Strawweight,3,unlisted,Orthodox,165.10,165.10,162.560000,167.640000,115.0,115.0
5140,Red,FALSE,Women's Flyweight,3,Orthodox,Orthodox,167.64,175.26,165.100000,172.720000,125.0,125.0


In [ ]:
# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100

In [72]:
fnf_df.dtypes

winner               object
title_bout           object
weight_class         object
no_of_rounds          int64
r_fighter_stance     object
b_fighter_stance     object
b_fighter_height    float64
r_fighter_height    float64
b_fighter_reach     float64
r_fighter_reach     float64
b_fighter_weight    float64
r_fighter_weight    float64
dtype: object

In [73]:
# Generate our categorical variable lists
app_cat = fnf_df.dtypes[fnf_df.dtypes == "object"].index.tolist()

In [74]:
app_cat

['winner',
 'title_bout',
 'weight_class',
 'r_fighter_stance',
 'b_fighter_stance']

In [75]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(fnf_df[app_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(app_cat)
encode_df.head()

,winner_Blue,winner_Draw,winner_Red,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,...,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [81]:
pd.options.display.max_columns = None
display(encode_df)

,winner_Blue,winner_Draw,winner_Red,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5138,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5139,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5140,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [82]:
fnf_df

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,183.289714,183.665789,216.0,175.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,183.665789,205.0,175.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,183.289714,183.665789,196.0,175.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,183.289714,183.665789,185.0,190.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,183.289714,183.665789,275.0,216.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5137,Blue,FALSE,Women's Strawweight,3,Orthodox,Orthodox,165.10,160.02,167.640000,162.560000,115.0,115.0
5138,Red,TRUE,Women's Flyweight,5,Southpaw,Orthodox,167.64,165.10,167.640000,167.640000,125.0,125.0
5139,Red,FALSE,Women's Strawweight,3,unlisted,Orthodox,165.10,165.10,162.560000,167.640000,115.0,115.0
5140,Red,FALSE,Women's Flyweight,3,Orthodox,Orthodox,167.64,175.26,165.100000,172.720000,125.0,125.0


In [88]:
encode_df.columns

Index(['winner_Blue', 'winner_Draw', 'winner_Red ', 'title_bout_FALSE',
       'title_bout_TRUE ', 'weight_class_Bantamweight',
       'weight_class_Catch Weight', 'weight_class_Featherweight',
       'weight_class_Flyweight', 'weight_class_Heavyweight',
       'weight_class_Light Heavyweight', 'weight_class_Lightweight',
       'weight_class_Middleweight', 'weight_class_Open Weight',
       'weight_class_Welterweight', 'weight_class_Women's Bantamweight',
       'weight_class_Women's Featherweight', 'weight_class_Women's Flyweight',
       'weight_class_Women's Strawweight', 'r_fighter_stance_Open Stance',
       'r_fighter_stance_Orthodox', 'r_fighter_stance_Sideways',
       'r_fighter_stance_Southpaw', 'r_fighter_stance_Switch',
       'r_fighter_stance_unlisted', 'b_fighter_stance_Open Stance',
       'b_fighter_stance_Orthodox', 'b_fighter_stance_Sideways',
       'b_fighter_stance_Southpaw', 'b_fighter_stance_Switch',
       'b_fighter_stance_unlisted'],
      dtype='object')

In [89]:
fnf_df_num = pd.merge(fnf_df, encode_df, on = ['winner', 'title_bout', 'weight_class', 'no_of_rounds', 'r_fighter_stance', 'b_fighter_stance', 'b_fighter_height', 'r_fighter_height', 'b_fighter_reach',
       'r_fighter_reach', 'b_fighter_weight', 'r_fighter_weight', 'winner_Blue', 'winner_Draw', 'winner_Red ', 'title_bout_FALSE',
       'title_bout_TRUE ', 'weight_class_Bantamweight',
       'weight_class_Catch Weight', 'weight_class_Featherweight',
       'weight_class_Flyweight', 'weight_class_Heavyweight',
       'weight_class_Light Heavyweight', 'weight_class_Lightweight',
       'weight_class_Middleweight', 'weight_class_Open Weight',
       'weight_class_Welterweight', "weight_class_Women's Bantamweight",
       "weight_class_Women's Featherweight", "weight_class_Women's Flyweight",
       "weight_class_Women's Strawweight", 'r_fighter_stance_Open Stance',
       'r_fighter_stance_Orthodox', 'r_fighter_stance_Sideways',
       'r_fighter_stance_Southpaw', 'r_fighter_stance_Switch',
       'r_fighter_stance_unlisted', 'b_fighter_stance_Open Stance',
       'b_fighter_stance_Orthodox', 'b_fighter_stance_Sideways',
       'b_fighter_stance_Southpaw', 'b_fighter_stance_Switch',
       'b_fighter_stance_unlisted'], how='outer')

KeyError: 'winner'

In [97]:
fnf_df_num = pd.concat([fnf_df,encode_df],axis=1)

In [98]:
fnf_df_num

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,winner_Blue,winner_Draw,winner_Red,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,183.289714,183.665789,216.0,175.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,183.665789,205.0,175.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,183.289714,183.665789,196.0,175.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,183.289714,183.665789,185.0,190.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,183.289714,183.665789,275.0,216.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,Blue,FALSE,Women's Strawweight,3,Orthodox,Orthodox,165.10,160.02,167.640000,162.560000,115.0,115.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5138,Red,TRUE,Women's Flyweight,5,Southpaw,Orthodox,167.64,165.10,167.640000,167.640000,125.0,125.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5139,Red,FALSE,Women's Strawweight,3,unlisted,Orthodox,165.10,165.10,162.560000,167.640000,115.0,115.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5140,Red,FALSE,Women's Flyweight,3,Orthodox,Orthodox,167.64,175.26,165.100000,172.720000,125.0,125.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


     winner title_bout         weight_class  no_of_rounds r_fighter_stance  \
0      Red       TRUE          Catch Weight             1         Southpaw   
1      Red       FALSE          Open Weight             1         Southpaw   
2      Red       FALSE          Open Weight             1         Southpaw   
3      Red       FALSE          Open Weight             1         Southpaw   
4      Red       FALSE          Open Weight             1         Orthodox   
...     ...        ...                  ...           ...              ...   
5137   Blue      FALSE  Women's Strawweight             3         Orthodox   
5138   Red       TRUE     Women's Flyweight             5         Southpaw   
5139   Red       FALSE  Women's Strawweight             3         unlisted   
5140   Red       FALSE    Women's Flyweight             3         Orthodox   
5141   Blue      FALSE          Heavyweight             3         Southpaw   

     b_fighter_stance  b_fighter_height  r_fighter_height  b_fi

,winner,title_bout,weight_class,no_of_rounds,r_fighter_stance,b_fighter_stance,b_fighter_height,r_fighter_height,b_fighter_reach,r_fighter_reach,b_fighter_weight,r_fighter_weight,winner_Blue,winner_Draw,winner_Red,title_bout_FALSE,title_bout_TRUE,weight_class_Bantamweight,weight_class_Catch Weight,weight_class_Featherweight,weight_class_Flyweight,weight_class_Heavyweight,weight_class_Light Heavyweight,weight_class_Lightweight,weight_class_Middleweight,weight_class_Open Weight,weight_class_Welterweight,weight_class_Women's Bantamweight,weight_class_Women's Featherweight,weight_class_Women's Flyweight,weight_class_Women's Strawweight,r_fighter_stance_Open Stance,r_fighter_stance_Orthodox,r_fighter_stance_Sideways,r_fighter_stance_Southpaw,r_fighter_stance_Switch,r_fighter_stance_unlisted,b_fighter_stance_Open Stance,b_fighter_stance_Orthodox,b_fighter_stance_Sideways,b_fighter_stance_Southpaw,b_fighter_stance_Switch,b_fighter_stance_unlisted
0,Red,TRUE,Catch Weight,1,Southpaw,Orthodox,195.58,185.42,183.289714,183.665789,216.0,175.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
1,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,182.880000,183.665789,205.0,175.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,Red,FALSE,Open Weight,1,Southpaw,Orthodox,185.42,185.42,183.289714,183.665789,196.0,175.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,Red,FALSE,Open Weight,1,Southpaw,unlisted,187.96,180.34,183.289714,183.665789,185.0,190.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,Red,FALSE,Open Weight,1,Orthodox,Orthodox,193.04,195.58,183.289714,183.665789,275.0,216.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5137,Blue,FALSE,Women's Strawweight,3,Orthodox,Orthodox,165.10,160.02,167.640000,162.560000,115.0,115.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5138,Red,TRUE,Women's Flyweight,5,Southpaw,Orthodox,167.64,165.10,167.640000,167.640000,125.0,125.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
5139,Red,FALSE,Women's Strawweight,3,unlisted,Orthodox,165.10,165.10,162.560000,167.640000,115.0,115.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
5140,Red,FALSE,Women's Flyweight,3,Orthodox,Orthodox,167.64,175.26,165.100000,172.720000,125.0,125.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
